In [1]:
# ensure packages are install
!pip install pandas tensorflow numpy chess matplotlib

^C


In [ ]:
# upload the dataset to colab, the path is defined here
!mkdir data
dataset_path = 'data/games.csv'

Ein Unterverzeichnis oder eine Datei mit dem Namen "data" existiert bereits.


trains heuristic chess function on all games that were won by resign or mate

In [ ]:
# make imports
import pandas as pd
import numpy as np
import chess
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.layers import BatchNormalization, Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

  Using cached tensorflow-2.4.1-cp38-cp38-win_amd64.whl (370.7 MB)


ERROR: Operation cancelled by user


In [ ]:
# ensure tensorflow is imported properly
print(tf.__version__)
tf.test.is_gpu_available()

In [ ]:
# utility functions for chess
chess_dict = {
    'p': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'P': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    'n': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'N': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    'b': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'B': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    'r': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    'R': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    'q': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    'Q': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    'k': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    'K': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    '.': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
}


def make_matrix(board):
    pgn = board.epd()
    foo = []
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append('.')
            else:
                foo2.append(thing)
        foo.append(foo2)
    return foo


def translate(matrix):
    rows = []
    for row in matrix:
        terms = []
        for term in row:
            terms.append(chess_dict[term])
        rows.append(terms)
    return rows

In [ ]:
# create the model and compile
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=1, activation='relu', input_shape=(8, 8, 12)))
model.add(MaxPooling2D())
model.add(Conv2D(filters=24, kernel_size=1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(filters=10, kernel_size=1, activation='relu'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(1, activation='tanh'))

model.compile(optimizer='Nadam', loss='mse')

In [ ]:
# read the pandas dataframe and create X and y
df = pd.read_csv(dataset_path, sep=';')
df = df[df['winner'] != 'draw']
df = df[df['victory_status'] != 'outoftime']
moves = df['moves'].values
winner = df['winner'].values
X = []
y = []

In [ ]:

# create dataset
index = 0
for game in moves:
    all_moves = game.split()
    total_moves = len(all_moves)
    if winner[index] == 'black':
        game_winner = -1
    else:
        game_winner = 1

    board = chess.Board()
    for i in range(len(all_moves)):
        board.push_san(all_moves[i])
        value = game_winner * (i / total_moves)
        matrix = make_matrix(board.copy())
        rows = translate(matrix)
        X.append([rows])
        y.append(value)

    index += 1

In [ ]:
# redefine X and y as numpy arrays (and reshape X)
X = np.array(X).reshape((len(X), 8, 8, 12))
y = np.array(y)

In [ ]:
# start training
# set name of files
h5 = 'model.h5'
json = 'model.json'

# create callbacks
checkpoint = keras.callbacks.ModelCheckpoint(h5,
                                             monitor='loss',
                                             verbose=0,
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='auto',
                                             period=1)
es = keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=500)
callback = [checkpoint, es]
model_json = model.to_json()

with open(json, "w") as json_file:
    json_file.write(model_json)

history = model.fit(X, y, epochs=1000, verbose=2, callbacks=callback)

# plot history
plt.plot(history.history['loss'])

In [ ]:
# convert model to tensorflow lite (optionally)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_model = converter.convert()
open("model.tflite", "wb").write(tf_model)